<a href="https://colab.research.google.com/github/LC1332/Prophet-Andrew-Ng/blob/main/langchain/%E6%9D%8E%E9%B2%81%E9%B2%81%E5%AD%A6LangChain_08_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 李鲁鲁学LangChain 08 Summarization

这是李鲁鲁的学习笔记，对Sam Witteveen的油管课程的记录

增加了李鲁鲁学习时候的吐槽

因为一个独立的md文件导入知乎会更方便，所以这个课程的笔记会记录在md中

比如这节课在

https://github.com/LC1332/Prophet-Andrew-Ng/blob/main/langchain/%E6%9D%8E%E9%B2%81%E9%B2%81%E5%AD%A6LangChain%2007.md

欢迎访问[骆驼项目](https://github.com/LC1332/Luotuo-Chinese-LLM)主页和点赞

In [1]:
!pip -q install langchain openai==0.27.0 tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.7/993.7 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.4 MB/s eta 0:00:00


# Summarization

History  
Challenges  
Fine-tuning  
Instruct Tuning

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "sk-KP8Zco"

In [3]:
!pip show langchain

Name: langchain
Version: 0.0.191
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


### Setting up Summarization Chain

In [4]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

# llm = OpenAI(temperature=0)

李鲁鲁:替换成3.5

In [37]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo')

In [25]:
text_splitter = CharacterTextSplitter(
    chunk_size = 1600,
    chunk_overlap  = 50)

我们把文章换个天龙八部吧

In [7]:
!wget https://raw.githubusercontent.com/LC1332/Prophet-Andrew-Ng/main/langchain/%E5%A4%A9%E9%BE%99%E5%85%AB%E9%83%A8%E7%89%87%E6%AE%B5.txt

--2023-06-06 05:34:11--  https://raw.githubusercontent.com/LC1332/Prophet-Andrew-Ng/main/langchain/%E5%A4%A9%E9%BE%99%E5%85%AB%E9%83%A8%E7%89%87%E6%AE%B5.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35333 (35K) [text/plain]
Saving to: ‘天龙八部片段.txt’

天龙八部片段.txt    100%[===================>]  34.50K  --.-KB/s    in 0.002s  

2023-06-06 05:34:12 (18.4 MB/s) - ‘天龙八部片段.txt’ saved [35333/35333]



In [26]:
# load the doc
with open('/content/天龙八部片段.txt') as f:
    demi_gods = f.read()
texts = text_splitter.split_text(demi_gods)

In [27]:
print(len(texts))
print(len(texts[-1]))

8
1151


In [31]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts]

In [ ]:
docs

##  3 types of CombineDocuments Chains

[Taken from the LangChain Docs](https://langchain.readthedocs.io/en/latest/modules/indexes/combine_docs.html)

## Summarize Simple with map_reduce

### Map Reduce
This method involves **an initial prompt on each chunk of data ***
( for summarization tasks, this could be a summary of that chunk; for question-answering tasks, it could be an answer based solely on that chunk). **Then a different prompt is run to combine all the initial outputs.** This is implemented in the LangChain as the MapReduceDocumentsChain.

**Pros:** Can scale to larger documents (and more documents) than StuffDocumentsChain. The calls to the LLM on individual documents are independent and can therefore be parallelized.

**Cons:** Requires many more calls to the LLM than StuffDocumentsChain. Loses some information during the final combining call.

In [29]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

In [22]:
chain = load_summarize_chain(llm, 
                             chain_type="map_reduce")


output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

Jiumozhi, a monk, arrives at Shaolin Temple and claims to have mastered all 72 of Shaolin's martial
arts techniques, which is met with skepticism. He impresses the high-ranking monks with his skills
and challenges them to disperse. Later, Xuzhu, a young monk, helps defend the temple against
Jiumozhi and engages in a fierce battle. Despite reaching a stalemate, Xu Zhu improves and counter-
attacks, causing concern among some of the monks. Jiumozhi resorts to using a knife but accidentally
stabs Xu Zhu in the end.


In [25]:
# for summarizing each part
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [26]:
# for combining the parts
chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

看看能不能换成中文

In [27]:
# for summarizing each part
chain.llm_chain.prompt.template = """对下列文本编写一个简明的总结:\n\n\n"{text}"\n\n\n总结:"""

In [28]:
# for combining the parts
chain.combine_document_chain.llm_chain.prompt.template = """对下列文本编写一个简明的总结:\n\n\n"{text}"\n\n\n总结:"""

In [29]:
chain = load_summarize_chain(llm, 
                             chain_type="map_reduce")


output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

鸠摩智以超凡口才和精湛武功惊艳了少林寺的众僧。他展示了自己的绝世武功，让高僧们叹服。虚竹在少林寺遇到了种种怪事，展示了高超的轮指封穴技巧。鸠摩智与虚竹展开激烈的武术对决，虚竹使用逍遥派的武学技巧，而鸠
摩智使用火焰刀等危险技巧。最终虚竹以反击和抓住鸠摩智的手腕为契机，战胜了鸠摩智。



鸠摩智以超凡口才和精湛武功惊艳了少林寺的众僧。他展示了自己的绝世武功，让高僧们叹服。虚竹在少林寺遇到了种种怪事，展示了高超的轮指封穴技巧。鸠摩智与虚竹展开激烈的武术对决，虚竹使用逍遥派的武学技巧，而鸠
摩智使用火焰刀等危险技巧。最终虚竹以反击和抓住鸠摩智的手腕为契机，战胜了鸠摩智。

In [ ]:
chain = load_summarize_chain(llm, 
                             chain_type="map_reduce",
                             verbose=True
                             )


output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, 
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

我保留一下sam原来的代码

In [ ]:
chain = load_summarize_chain(llm, 
                             chain_type="map_reduce",
                             verbose=True
                             )


output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, 
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
Write a concise summary of the following:


"How This Book Was Written-And Why 
by 

Dale Carnegie 

During the first thirty-five years of the twentieth century, the 
publishing houses of America printed more than a fifth of a million 
different books. Most of them were deadly dull, and many were 
financial failures. "Many," did I say? The president of one of the 
largest publishing houses in the world confessed to me that his 
company, after seventy-five years of publishing experience, still lost 
money on seven out of every eight books it published. 

Why, then, did I have the temerity to write another book? And, after 
I had written it, why should you bother to read it? 


Fair questions, both; and I'll try to answer them. 


I have, since 1912, been conducting educational courses for business 
and professional men and women in New York. At first, I conducted 
courses in public speaking only - courses designe

### Summarizing with the 'stuff' Chain



### Stuffing
Stuffing is the simplest method, whereby you simply stuff all the related data into the prompt as context to pass to the language model. This is implemented in LangChain as the StuffDocumentsChain.

**Pros:** Only makes a single call to the LLM. When generating text, the LLM has access to all the data at once.

**Cons:** Most LLMs have a context length, and for large documents (or many documents) this will not work as it will result in a prompt larger than the context length.

The main downside of this method is that **it only works one smaller pieces of data.**  Once you are working with many pieces of data, this approach is no longer feasible. The next two approaches are designed to help deal with that.



In [32]:
chain = load_summarize_chain(llm, chain_type="stuff")

# for summarizing each part
chain.llm_chain.prompt.template = """对下列文本编写一个简明的总结:\n\n\n"{text}"\n\n\n总结:"""

In [36]:
print(chain.document_prompt)

input_variables=['page_content'] output_parser=None partial_variables={} template='{page_content}' template_format='f-string' validate_template=True


In [43]:
prompt_template = """对以下内容进行简明的要点总结，用bullet point形式:


{text}


使用BULLET POINTS形式总结:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["text"])


In [41]:
chain = load_summarize_chain(llm, 
                             chain_type="stuff", 
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs[:1])

wrapped_text = textwrap.fill(output_summary, 
                             width=100,
                             break_long_words=True,
                             replace_whitespace=False)
print(wrapped_text)

- A clear and calm voice is heard outside the temple, asking to join the discussion on martial arts
at Shaolin Temple.
- The voice belongs to a monk named Ji Mo Zhi, who is introduced as a respected
figure from Tibet.
- Ji Mo Zhi claims to have mastered all 72 Shaolin martial arts, which is
considered impossible by the other monks present.
- Ji Mo Zhi demonstrates his skills by using the
"Big Golden Fist" and the "Diamond Palm" techniques, impressing the monks with his precision and
power.
- Ji Mo Zhi claims that he learned the Shaolin martial arts through his own efforts and
practice, not by stealing them from the Shaolin Temple.
- The other monks are skeptical of Ji Mo
Zhi's claims, but are also impressed by his skills.
- The discussion on martial arts continues, with
Ji Mo Zhi offering his insights and opinions.


### Ver 3 With 'map_reduce' with our custom prompt

In [45]:
chain = load_summarize_chain(llm, 
                             chain_type="map_reduce",
                             map_prompt=BULLET_POINT_PROMPT, 
                             combine_prompt=BULLET_POINT_PROMPT)

# chain.llm_chain.prompt= BULLET_POINT_PROMPT
# chain.combine_document_chain.llm_chain.prompt= BULLET_POINT_PROMPT

output_summary = chain.run(docs[:])
wrapped_text = textwrap.fill(output_summary, 
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

- 一个声音请求成为旁听者，自称吐蕃国师大轮明王，被邀请进来
- 鸠摩智认为玄生大师和哲罗星大师的武功有不足之处，群僧惊异，玄生请他指点开示
-
鸠摩智展现出了超越常人的武功实力，使用的招数叫做“三入地狱”和袈裟伏魔功
- 玄慈方丈认为虚竹武功精奇，可以与鸠摩智匹敌，决定让虚竹出去抵挡一阵
-
鸠摩智出般若掌，虚竹出韦陀掌，两者是少林武功的两个极端，两人开始战斗
- 虚竹使用北冥真气护体，鸠摩智使用多罗指和燃木刀法，两人近身肉搏，掌力强劲
-
为维护少林一派声誉，不志在杀人，最终虚竹使用逍遥派武学招数，鸠摩智使用匕首偷袭，旁观群僧对此不满


In [40]:
# with a custom prompt
prompt_template = """对以下内容进行简明的要点总结，用bullet points形式:


{text}


使用BULLET POINTS形式总结:"""

PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["text"])



llm = ChatOpenAI(model_name='gpt-3.5-turbo')

## with intermediate steps
chain = load_summarize_chain(llm, 
                             chain_type="map_reduce", 
                             return_intermediate_steps=True, 
                             map_prompt=PROMPT, 
                             combine_prompt=PROMPT)

output_summary = chain({"input_documents": docs}, return_only_outputs=True)
wrapped_text = textwrap.fill(output_summary['output_text'], 
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

- 鸠摩智展示少林派七十二门绝技中的问题和自己的武功
- 虚竹目睹鸠摩智的武功表演，展示了自己的医术和武功
- 群僧对鸠摩智和虚竹的武功和行为产生疑问
- 主人公来到少林寺，与虚竹进行一场较量
-
两人使用少林派基础武功和七十二绝技进行交手
- 虚竹最终使用天山六阳掌化解鸠摩智的攻击
- 两人进行近身肉搏，众高僧生平从未见过这样的打法
- 虚竹最终掌握天山六阳掌的精妙，得到少林群僧的欢喜
-
神山上人和波罗星也有自己的武学追求
- 虚竹使用逍遥派武学的狠辣招数，引起旁观者惊惧
- 鸠摩智使用匕首偷袭虚竹，被谴责


In [41]:
wrapped_text = textwrap.fill(output_summary['intermediate_steps'][2], 
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

- 虚竹目睹鸠摩智的武功表演
- 鸠摩智使用“小无相功”掌控其他武功
- “小无相功”是道家武学，与佛教武功不同
- 众僧虽有疑虑，但不了解“小无相功”的威力和特点
-
虚竹想指出鸠摩智并非使用少林派绝技，但一直沉默不语
- 鸠摩智建议少林寺解散，引起众僧不满
- 鸠摩智意在瓦解少林寺，取得好处


In [43]:
for inter_result in output_summary['intermediate_steps']:
    print(inter_result)

• 一个清朗的声音传来，是吐蕃国山僧鸠摩智，参见少林寺方丈。
• 群僧对他的身手感到惊异，他坐在神山的上首。
• 鸠摩智指出少林派七十二门绝技中的问题，包括每年能成一项绝技的说法。
• 群僧不理解他的意思，玄生请他解释。
• 鸠摩智展示了大金刚拳法中的一招“洛钟东应”，展示出了深得“大金刚拳”的秘要。
• 群僧对他的能力感到惊叹，但有些人仍持怀疑态度。
- 鸠摩智使用般若掌中的“慑伏外道”掌法，将铜鼎上的一块手掌般的物件割落
- 众僧惊叹他的“三入地狱”摩诃指功，认为他的功力已超凡入圣的境地
- 落下的物件是一只黄铜手掌，袍袖一拂后，他展示了“袈裟伏魔功”，玄慈方丈对其赞叹不已
- 玄慈方丈认为鸠摩智武功高极，少林派技不如人，将波罗星请离寺院
- 众僧心生失落，感受到少林派在中土武学中的失位，玄慈方丈也深思熟虑，认为再扣留波罗星已无益
- 鸠摩智的绝世神功让整个大殿寂静无声，震慑众人
- 虚竹目睹鸠摩智的武功表演
- 鸠摩智使用“小无相功”掌控其他武功
- “小无相功”是道家武学，与佛教武功不同
- 众僧虽有疑虑，但不了解“小无相功”的威力和特点
- 虚竹想指出鸠摩智并非使用少林派绝技，但一直沉默不语
- 鸠摩智建议少林寺解散，引起众僧不满
- 鸠摩智意在瓦解少林寺，取得好处
- 一位僧人称赞大理天龙寺，玄渡大师受挫
- 虚竹救治玄渡大师，展示高超医术
- 群僧对虚竹的武功和行为产生疑问
- 庄聚贤要求少林派承认其为中原武林盟主，众高僧为此烦恼
- 虚竹师父慧轮在园中偷偷挑粪浇菜，众高僧不知
- 群僧对鸠摩智的指法产生质疑，鸠摩智恍然大悟
- 主人公是一个天生聪明、运气好的人，从小就遇到奇遇，从未败过。
- 他来到少林寺，想单枪匹马打败这座千年古刹。
- 他和虚竹进行了一场较量，虚竹使用了玄渡太师伯的拈花指和小无相功。
- 鸠摩智试了虚竹一招般若掌，虚竹用天山六阳掌将其化解。
- 鸠摩智要求虚竹使用少林派的功夫，虚竹只会罗汉拳和韦陀掌。
- 方丈让虚竹与鸠摩智过招，虚竹使用了韦陀掌的起手式灵山礼佛。
- 鸠摩智使用般若掌的上乘功夫，但最终不得不避战。
- 韦陀掌是少林派的基础武功
- 学韦陀掌后，可学习般若掌，需要数十年练习
- 般若掌是少林派七十二绝技之一，无穷无尽
- 鸠摩智与虚竹交手，使用多种少林派神功，虚竹只使用“黑虎偷心”一招
- “黑虎偷心”虽拙劣，但劲

### With the 'refine' CombineDocument Chain

## Refine
This method involves **an initial prompt on the first chunk of data, generating some output. For the remaining documents, that output is passed in, along with the next document**, asking the LLM to refine the output based on the new document.

**Pros:** Can pull in more relevant context, and may be less lossy than MapReduceDocumentsChain.

**Cons:** Requires many more calls to the LLM than StuffDocumentsChain. The calls are also NOT independent, meaning they cannot be paralleled like MapReduceDocumentsChain. There is also some potential dependencies on the ordering of the documents.

In [ ]:
chain = load_summarize_chain(llm, chain_type="refine")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

  Dale Carnegie wrote this book to provide practical, common-sense training to adults in how to
effectively deal with people in everyday business and social contacts. He has been conducting
educational courses for business and professional men and women since 1912 and has found that even
in technical lines such as engineering, 85% of one's financial success is due to skill in human
engineering. He believes that every college should provide courses to develop this ability, but has
not found one that does. To create this book, Carnegie conducted a survey to determine what adults
wanted to study, read extensively on the subject, and interviewed successful people to discover the
techniques they used in human relations. The book was developed out of the experiences of thousands
of adults in a laboratory of human relationships, the first of its kind. Carnegie's book provides
practical advice on how to revolutionize one's life and relationships through the application of his
principles. He ha

In [ ]:
prompt_template = """Write a concise summary of the following extracting the key information:


{text}


CONCISE SUMMARY:"""
PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["text"])

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
chain = load_summarize_chain(OpenAI(temperature=0), 
                             chain_type="refine", 
                             return_intermediate_steps=True, 
                             question_prompt=PROMPT, 
                             refine_prompt=refine_prompt)


In [45]:
prompt_template = """提取关键信息，写出以下内容的简明概述:


{text}


简明总结:"""

llm = ChatOpenAI(model_name='gpt-3.5-turbo')


PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["text"])

refine_template = (
    "您的工作是生成一个最终的总结\n"
    "我们已经提供了一个前半部分的现有的总结：{existing_answer}\n"
    "我们将通过增量的一些段落的上下文来补充现有的总结(只有在需要的情况下)。\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "根据新的上下文，完善原始的总结"
    "如果这个上下文不相关，可以返回原始的总结。"
)

refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
chain = load_summarize_chain(llm, 
                             chain_type="refine", 
                             return_intermediate_steps=True, 
                             question_prompt=PROMPT, 
                             refine_prompt=refine_prompt)


In [ ]:
output_summary = chain({"input_documents": docs}, return_only_outputs=True)


In [51]:
wrapped_text = textwrap.fill(output_summary['output_text'], 
                             width=35,
                             break_long_words=True,
                             replace_whitespace=False)
print(wrapped_text)

在这场激烈的战斗中，虚竹和鸠摩智的武艺都得到了提升。虚竹通过领悟天山六
阳掌的精髓，开始有了反击的能力，而鸠摩智的攻势也因此略有顿挫。随着时间
的推移，虚竹越来越能够反击，少林群僧也因此欣喜。他们并不是想杀死鸠摩智
，而是为了维护少林派的声誉。虽然虚竹的天山六阳掌和鸠摩智的少林派武功形
成了僵持之局，但是虚竹逐渐掌握了进攻的主动权，让众人惊叹。在这个故事中
，武学、医学、宗教和政治相互交织，一个人的道德和信仰如何能够影响整个社
会。最终，虚竹成为了少林寺的一位神僧，而鸠摩智也为自己的行为所付出了代
价。然而，这场战斗也让人思考，对于武学的探索和研究，到底是为了何种目的
，是为了更高级的武功还是为了自己的私欲？这是值得深思的问题。


In [58]:
wrapped_text = textwrap.fill(output_summary['intermediate_steps'][2], 
                             width=35,
                             break_long_words=True,
                             replace_whitespace=False)
print(wrapped_text)

鸠摩智展示出惊人的武功，让少林寺的众僧大受震撼。虚竹心中有疑惑，发现鸠
摩智所使用的各种招式均是运用他所掌握的“小无相功”，而非少林派绝技。然
而，少林寺中无人能够看出这一点，因为“小无相功”是道家的武学，而少林弟
子都是佛门弟子。在鸠摩智的压迫下，少林寺的高僧们不得不让波罗星离开寺庙
。最终，鸠摩智提出将少林寺散掉的建议，少林群僧愤然反驳，但他们也意识到
鸠摩智要以一人之力将少林寺挑了，这对于吐蕃国大有好处。在众人惊叹的目光
下，鸠摩智成为了少林寺的一位神僧。
